# Trading Project

- Using yahoo api to get the price from different companies

In [75]:
import yfinance as yf
import pandas as pd

In [2]:
tickerSymbol = "AAPL"
tickerData = yf.Ticker(tickerSymbol)
apple = tickerData.history(period='1d', start='2010-1-1', end="2020-9-6")
tickerData.recommendations

,Firm,To Grade,From Grade,Action
Date,,,,
2012-02-09 08:17:00,Canaccord Genuity,Buy,,main
2012-03-08 07:33:00,FBN Securities,Outperform,,main
2012-03-13 06:08:00,Jefferies,Buy,,main
2012-03-14 06:13:00,Morgan Stanley,Overweight,,main
2012-03-14 08:21:00,Canaccord Genuity,Buy,,main
...,...,...,...,...
2020-08-31 12:07:36,Wedbush,Outperform,,main
2020-08-31 16:24:32,Argus Research,Buy,,main
2020-09-01 12:50:49,Bank of America,Neutral,,reit


In [76]:
import datetime

start_sp = datetime.datetime(2010, 1, 1)
end_sp = datetime.datetime(2020, 9, 7)

In [88]:
from pandas_datareader import data as pdr
#import fix_yahoo_finance as yf
yf.pdr_override() # <== that's all it takes :-)
SP500 = pdr.get_data_yahoo('SPX', start_sp, end_sp)
    
SP500.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2009-12-31,0.05159,0.05159,0.04643,0.04643,0.04643,0
2010-01-04,0.05159,0.05159,0.05159,0.05159,0.05159,193846
2010-01-05,0.05159,0.06448,0.05159,0.06190,0.06190,1125544
2010-01-06,0.05933,0.05933,0.05933,0.05933,0.05933,116308
2010-01-07,0.05417,0.05417,0.05417,0.05417,0.05417,36055


In [89]:
len(SP500)#2689

2017

### Using Mongodb

In [98]:
import pymongo
from pymongo import MongoClient

In [79]:
myclient = MongoClient("mongodb://localhost:27017/")
mydb = myclient["Stocks"]
mycol = mydb["Tickers"]

In [90]:
# Step 2: Insert Data into DB
SP500.reset_index(inplace=True) # Reset Index
data_dict = SP500.to_dict("records") # Convert to dictionary
mycol.insert_one({"Index":"SPX","data":data_dict}) # inesrt into DB

### Scraping News for Sentiment Analysis

In [91]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
# NLTK VADER for sentiment analysis
#from nltk.sentiment.vader import SentimentIntensityAnalyzer

finwiz_url = 'https://finviz.com/quote.ashx?t='

In [92]:
news_tables = {}
tickers = ['AMZN', 'TSLA', 'GOOG', 'AAPL', 'MSFT']

for ticker in tickers:
    url = finwiz_url + ticker
    req = Request(url=url,headers={'user-agent': 'my-app/0.0.1'}) 
    response = urlopen(req)    
    # Read the contents of the file into 'html'
    html = BeautifulSoup(response)
    # Find 'news-table' in the Soup and load it into 'news_table'
    news_table = html.find(id='news-table')
    # Add the table to our dictionary
    news_tables[ticker] = news_table

In [93]:
# Read one single day of headlines for 'AMZN' 
amzn = news_tables['AMZN']
# Get all the table rows tagged in HTML with <tr> into 'amzn_tr'
amzn_tr = amzn.findAll('tr')

for i, table_row in enumerate(amzn_tr):
    # Read the text of the element 'a' into 'link_text'
    a_text = table_row.a.text
    # Read the text of the element 'td' into 'data_text'
    td_text = table_row.td.text
    # Print the contents of 'link_text' and 'data_text' 
    print(a_text)
    print(td_text)
    # Exit after printing 4 rows of data
    if i == 3:
        break

SoftBank Selloff Hits $17 Billion After Tech Exposure Fear Grows
Sep-08-20 10:07PM  
Go Ahead and Call Warren Buffett a Snowflake
06:53PM  
Warren Buffett and Salesforce Will Invest in Snowflake's IPO
06:26PM  
Stock market news live updates: Stock futures fall further after S&P 500's lowest close in a month
06:15PM  


In [94]:
parsed_news = []

# Iterate through the news
for file_name, news_table in news_tables.items():
    # Iterate through all tr tags in 'news_table'
    for x in news_table.findAll('tr'):
        # read the text from each tr tag into text
        # get text from a only
        text = x.a.get_text() 
        # splite text in the td tag into a list 
        date_scrape = x.td.text.split()
        # if the length of 'date_scrape' is 1, load 'time' as the only element

        if len(date_scrape) == 1:
            time = date_scrape[0]
            
        # else load 'date' as the 1st element and 'time' as the second    
        else:
            date = date_scrape[0]
            time = date_scrape[1]
        # Extract the ticker from the file name, get the string up to the 1st '_'  
        ticker = file_name.split('_')[0]
        
        # Append ticker, date, time and headline as a list to the 'parsed_news' list
        parsed_news.append([ticker, date, time, text])
        
parsed_news

[['AMZN',
  'Sep-08-20',
  '10:07PM',
  'SoftBank Selloff Hits $17 Billion After Tech Exposure Fear Grows'],
 ['AMZN',
  'Sep-08-20',
  '06:53PM',
  'Go Ahead and Call Warren Buffett a Snowflake'],
 ['AMZN',
  'Sep-08-20',
  '06:26PM',
  "Warren Buffett and Salesforce Will Invest in Snowflake's IPO"],
 ['AMZN',
  'Sep-08-20',
  '06:15PM',
  "Stock market news live updates: Stock futures fall further after S&P 500's lowest close in a month"],
 ['AMZN',
  'Sep-08-20',
  '04:54PM',
  'Dow closes down 632 points, as tech rout leaves Nasdaq in correction territory'],
 ['AMZN',
  'Sep-08-20',
  '04:53PM',
  '2 reasons fears of a dot-com style tech bubble are overblown'],
 ['AMZN',
  'Sep-08-20',
  '04:29PM',
  'US STOCKS-Stocks end lower as tech swoon persists, Tesla in historic drop'],
 ['AMZN',
  'Sep-08-20',
  '04:26PM',
  '3M Co. stock falls Tuesday, still outperforms market'],
 ['AMZN',
  'Sep-08-20',
  '04:25PM',
  'Netflix Inc. stock falls Tuesday, still outperforms market'],
 ['AMZN'

In [97]:
mycol.insert_one({"name":"News","data":parsed_news}) # inesrt into DB